In [ ]:
%load_ext autoreload

import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize

import pyOptLE as optle

In [ ]:
%autoreload

In [ ]:
generate = False
dt = 0.1
RT = 1.
beta = 1/RT

n_knots = 20

##############################

if generate:

    n = 50   # num traj
    N = 1000 # traj length
    D = .1
    
    k = 10.0
    def V(q):
        return 0.5 * k * (q**2)
    def nablaV(q):
        return k * q
    
    np.random.seed(1)

    # Start out of equilibrium
    q = np.zeros([n, N])

    q[:,0] = np.zeros(n) + np.random.normal(0, np.sqrt(2.*D*dt), size=n)
    for t in range(1,N):
        q[:,t] = q[:,t-1] - beta * D * nablaV(q[:,t-1]) * dt + np.random.normal(0, np.sqrt(2.*D*dt), size=n)


if not generate:

    # format data from "real" simulation
    raw = np.loadtxt('data/dblwl_abmd/biased/dblwl_h7_k05_g10_abmd')
    
    t = 0
    old_t = 0
    q = list()
    f = list()
    qi = list()
    fi = list()

    # N = list()
    for r in raw:
        if r[0] < old_t:
            # print(i)
            # N.append(i)
            q.append(np.array(qi))
            f.append(np.array(fi))
            qi = list()
            fi = list()

        qi.append(r[1])
        fi.append(r[2])
        old_t = r[0]

    plt.plot([qi.shape[0] for qi in q])
    # q = raw[:N*n,1].reshape([N, n])

In [ ]:
# Precompute displacements
deltaq = [qi[1:] - qi[:-1] for qi in q]
# deltaq = np.array(deltaq, dtype=object)

epsilon = 1e-10
# Add margin for floating-point precision
qmin = min([np.min(qi) for qi in q]) - epsilon
qmax = max([np.max(qi) for qi in q]) + epsilon

# import multiprocessing
# num_processes = multiprocessing.cpu_count()

knots = np.linspace(qmin, qmax, n_knots)

# Initial guess for the parameters
D = .1
initial_params = np.concatenate((np.zeros(n_knots), np.zeros(n_knots) + D))


In [ ]:
initial_params

In [ ]:
start_time = time.time()
print(optle.objective(initial_params, q, deltaq, f, knots))
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print(optle.objective(initial_params, q, deltaq, f, knots))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:

def history(res):
    history.opt.append(res)
    if history.nsteps%100 == 0:
        print('Step', history.nsteps, np.square(res).sum())
    history.nsteps += 1

history.opt = list()
history.nsteps = 0

start_time = time.time()
# Minimize the objective function using L-BFGS-B algorithm
result = minimize(optle.objective, initial_params,  args=(q, deltaq, f, knots), jac=True, method='L-BFGS-B', callback=history)
print("--- %s seconds ---" % (time.time() - start_time))

print(result.nit, result.message)

In [ ]:
optimized_params = result.x

# Extract the optimized knots and coefficients
optimized_knots = knots
optimized_G = optimized_params[:n_knots]
optimized_logD = optimized_params[n_knots:]

interp_factor = 10

# Fencepost theorem aplied forward and then backward
n_discr_points = interp_factor * (len(knots) - 1) + 1
x = np.linspace(qmin, qmax, n_discr_points)


%matplotlib inline
# Plot the original data and the optimized spline
import matplotlib.pyplot as plt

counts, bins = np.histogram(np.concatenate(q), bins=40)
# FE = -RT * np.log(counts)
# FE -= np.min(FE)
# plt.stairs(FE, bins, label='FE from histogram')
plt.stairs(-RT*(np.log(counts)-np.max(np.log(counts))), bins, label='\"Free energy\" from histogram')


# TODO reuse this for ABF
# Integral of linear interpolation

predicted = optle.piecewise_linear_int(x, knots, optimized_G)
plt.plot(x, predicted, label='Optimized free energy')
plt.xlabel('colvar q')
plt.ylabel('Free energy')
plt.legend()

if generate:
    plt.scatter(x, V(x), label='True potential')
    plt.legend()

plt.figure()
plt.plot(x, np.exp(np.interp(x, knots, optimized_logD)), label='Optimized D')
if generate:
    plt.plot([qmin, qmax], [D, D], label='True D')
plt.xlabel('colvar q')
plt.ylabel('Diffusion coefficient')
plt.legend()


plot_gradient = False
if plot_gradient:
    plt.figure()
    predicted_nablaV = np.interp(x, knots, optimized_G)
    plt.scatter(knots, optimized_G, label='Optimized grad G')
    plt.plot(x, predicted_nablaV, label='Optimized gradient')
    if generate:
        plt.plot(x, np.interp(x, knots, nablaV(knots)), label='True gradient')
    plt.xlabel('colvar q')
    plt.ylabel('Diffusion coefficient')
    plt.legend()


In [ ]:
no_bias = [np.zeros_like(fi) for fi in f]


history.opt = list()
history.nsteps = 0

start_time = time.time()
# Minimize the objective function using L-BFGS-B algorithm
result = minimize(optle.objective, initial_params,  args=(q, deltaq, no_bias, knots), jac=True, method='L-BFGS-B', callback=history)
print("--- %s seconds ---" % (time.time() - start_time))

print(result.nit, result.message)

In [ ]:
optimized_params = result.x

# Extract the optimized knots and coefficients
optimized_knots = knots
optimized_G = optimized_params[:n_knots]
optimized_logD = optimized_params[n_knots:]

interp_factor = 10

# Fencepost theorem aplied forward and then backward
n_discr_points = interp_factor * (len(knots) - 1) + 1
x = np.linspace(qmin, qmax, n_discr_points)


%matplotlib inline
# Plot the original data and the optimized spline
import matplotlib.pyplot as plt

counts, bins = np.histogram(np.concatenate(q), bins=40)
# FE = -RT * np.log(counts)
# FE -= np.min(FE)
# plt.stairs(FE, bins, label='FE from histogram')
plt.stairs(-RT*(np.log(counts)-np.max(np.log(counts))), bins, label='\"Free energy\" from histogram')


# TODO reuse this for ABF
# Integral of linear interpolation

predicted = optle.piecewise_linear_int(x, knots, optimized_G)
plt.plot(x, predicted, label='Optimized free energy')
plt.xlabel('colvar q')
plt.ylabel('Free energy')
plt.legend()

if generate:
    plt.scatter(x, V(x), label='True potential')
    plt.legend()

plt.figure()
plt.plot(x, np.exp(np.interp(x, knots, optimized_logD)), label='Optimized D')
if generate:
    plt.plot([qmin, qmax], [D, D], label='True D')
plt.xlabel('colvar q')
plt.ylabel('Diffusion coefficient')
plt.legend()


plot_gradient = False
if plot_gradient:
    plt.figure()
    predicted_nablaV = np.interp(x, knots, optimized_G)
    plt.scatter(knots, optimized_G, label='Optimized grad G')
    plt.plot(x, predicted_nablaV, label='Optimized gradient')
    if generate:
        plt.plot(x, np.interp(x, knots, nablaV(knots)), label='True gradient')
    plt.xlabel('colvar q')
    plt.ylabel('Diffusion coefficient')
    plt.legend()
